<a href="https://colab.research.google.com/github/daaaanish17/Image-Classification/blob/main/Animal_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Animal Classification using Transfer Learning Model (MobileNetV2)*

Image Dataset Link: https://www.kaggle.com/datasets/alessiocorrado99/animals10

Importing Libraries

In [ ]:
import os
import tensorflow as tf
import numpy as np
from google.colab import drive
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Conv2D, GlobalAvgPool2D, Input
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator

Connecting to Google Drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In my Google Drive I have already created a "animal_classification" folder and in this folder I have uploaded my kaggle dataset folder in zip format. 

In [ ]:
%cd drive/MyDrive/animal_classification/

/content/drive/MyDrive/animal_classification


In [ ]:
# unzipping archivee folder(image folder)

!unzip archivee.zipy

Using Python Dictionary present in animal_classification folder for renaming folders

In [ ]:
from translate import translate

translate

{'cane': 'dog',
 'cavallo': 'horse',
 'elefante': 'elephant',
 'farfalla': 'butterfly',
 'gallina': 'chicken',
 'gatto': 'cat',
 'mucca': 'cow',
 'pecora': 'sheep',
 'scoiattolo': 'squirrel',
 'dog': 'cane',
 'elephant': 'elefante',
 'butterfly': 'farfalla',
 'chicken': 'gallina',
 'cat': 'gatto',
 'cow': 'mucca',
 'spider': 'ragno',
 'squirrel': 'scoiattolo'}

Renaming folder names

In [ ]:
for i in os.listdir('raw-img'):
  os.rename('raw-img/' + i, 'raw-img/' + translate[i])

# os.rename('raw-img/ragno',  'raw-img/spider')  

In [ ]:
os.rename('raw-img/ragno',  'raw-img/spider')  
os.rename('raw-img/scoiattolo',  'raw-img/squirrel')

In [ ]:
os.rename('raw-img/gatto',  'raw-img/cat')

Displaying folder names

In [ ]:
for i in os.listdir('raw-img'):
  print(i)

dog
horse
elephant
butterfly
chicken
cat


Displaying No. of Images in each folder

In [ ]:
for i in os.listdir('raw-img'):
  print(i, len(os.listdir('raw-img/' + i)))

dog 4863
horse 2623
elephant 1446
butterfly 2112
chicken 3098
cat 1688


Creating new directories

And Entering data in new directories

In [ ]:
os.mkdir('train')
os.mkdir('test')

for i in os.listdir('raw-img'):
  os.mkdir('train/' + i)
  os.mkdir('test/' + i)

  for j in os.listdir('raw-img/'+i)[:1000]:    
    os.rename('raw-img/'+i+'/'+j, 'train/'+i+'/'+j)
  for j in os.listdir('raw-img/'+i)[:400]:
    os.rename('raw-img/'+i+'/'+j, 'test/'+i+'/'+j)

  #train dataset contains 1000 images while test dataset contains 400 images  

Creating function for Image Data Generator.

In [ ]:
def img_data(dir_path, target_size, batch, class_lst, pre_processs):
  if pre_processs:
    generate_object = ImageDataGenerator(preprocessing_function=pre_processs)
  else:
    generate_object = ImageDataGenerator()
   
  return (generate_object.flow_from_directory(dir_path, target_size=target_size, batch_size=batch,  #using flow_from_directory because all of our data is saved in form of directories.
                                              class_mode='sparse', classes=class_lst, shuffle=True))  

In [ ]:
train_data_gen = img_data('train', (224, 224), 500, os.listdir('train'), preprocess_input)
# mobilenet. preprocess_input will scale input pixels between -1 and 1. input_shape: Optional shape tuple, 
# only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3) 
# (with channels_last data format) or (3, 224, 224) (with channels_first data format).

valid_data_gen = img_data('test', (224, 224), 500, os.listdir('test'), preprocess_input)

Found 6000 images belonging to 6 classes.
Found 2400 images belonging to 6 classes.


Transfer Learning With MobileNet V2. MobileNet V2 model was developed at Google, pre-trained on the ImageNet dataset with 1.4M images and 1000 classes of web images. *We will use this as our base model* to train with our dataset and classify the images of animals

In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224, 224, 3),
    alpha=1.0,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
)

Freeze the convolutional base

It is important to freeze the convolutional base before you compile and train the model. Freezing (by setting layer.trainable = False) prevents the weights in a given layer from being updated during training. MobileNet V2 has many layers, so setting the entire model's trainable flag to False will freeze all of them.

In [ ]:
base_model.trainable=False

Creating our Image Classification Model

In [ ]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# This callback will stop the training when there is no improvement in # the loss for three consecutive epochs.

elst = callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [ ]:
# ModelCheckpoint callback allows you to continually save the model both during and at the end of training. 
# Model will saved in our drive ('MyDrive/animal_classification')

save_ck = callbacks.ModelCheckpoint('.mdl_wt.hdf5', save_best_only=True, monitor='val_loss', mode='min') 

In [ ]:
model.fit(train_data_gen, batch_size=500, validation_data=valid_data_gen, callbacks=[elst, save_ck], epochs=10)

Epoch 1/10
12/12 [==============================] - 5281s 447s/step - loss: 0.5057 - accuracy: 0.8192 - val_loss: 0.1670 - val_accuracy: 0.9513
Epoch 2/10
12/12 [==============================] - 42s 4s/step - loss: 0.0970 - accuracy: 0.9703 - val_loss: 0.1346 - val_accuracy: 0.9567
Epoch 3/10
12/12 [==============================] - 41s 3s/step - loss: 0.0579 - accuracy: 0.9813 - val_loss: 0.0936 - val_accuracy: 0.9704
Epoch 4/10
12/12 [==============================] - 42s 4s/step - loss: 0.0381 - accuracy: 0.9877 - val_loss: 0.0912 - val_accuracy: 0.9708
Epoch 5/10
12/12 [==============================] - 42s 3s/step - loss: 0.0236 - accuracy: 0.9945 - val_loss: 0.0900 - val_accuracy: 0.9708
Epoch 6/10
12/12 [==============================] - 43s 4s/step - loss: 0.0161 - accuracy: 0.9980 - val_loss: 0.0859 - val_accuracy: 0.9717
Epoch 7/10
12/12 [==============================] - 40s 3s/step - loss: 0.0120 - accuracy: 0.9990 - val_loss: 0.0870 - val_accuracy: 0.9708
Epoch 8/10
12/12